In [ ]:
import numpy as np
import torch
import joblib
import xgboost as xgb
import time
from sklearn.preprocessing import StandardScaler

# === Paths ===
dataset_name = "DIS_lab_LoS_16"  # example
model_name = "xgboost"
base_path = f"./LAMDA-TALENT/data"

# Load test datasets for Position X and Y
X_test_x = np.load(f"{base_path}/{dataset_name}_X/N_test.npy")
X_test_y = np.load(f"{base_path}/{dataset_name}_Y/N_test.npy")

# Load train datasets for Position X and Y
X_train_x = np.load(f"{base_path}/{dataset_name}_X/N_train.npy")
X_train_y = np.load(f"{base_path}/{dataset_name}_Y/N_train.npy")

# === Normalize features Position X ===
scaler = StandardScaler()
X_train_x = scaler.fit_transform(X_train_x)
X_test_x = scaler.transform(X_test_x)  # only transform test with train stats

# === Normalize features Position Y===
scaler = StandardScaler()
X_train_y = scaler.fit_transform(X_train_y)
X_test_y = scaler.transform(X_test_y)  # only transform test with train stats

# Load target Position X and Y
y_test_x = np.load(f"{base_path}/{dataset_name}_X/y_test.npy")
y_test_y = np.load(f"{base_path}/{dataset_name}_Y/y_test.npy")  # for scaler

# Load target Position X and Y
y_train_x = np.load(f"{base_path}/{dataset_name}_X/y_train.npy")
y_train_y = np.load(f"{base_path}/{dataset_name}_Y/y_train.npy")  # for scaler


mean_x, std_x = y_train_x.mean(), y_train_x.std()
mean_y, std_y = y_train_y.mean(), y_train_y.std()

# === Load models ===
x_model_path = f"./LAMDA-TALENT/LAMDA_TALENT/results_model/{dataset_name}_X-{model_name}-Tune/Norm-standard-Nan-mean-new-Cat-ordinal/best-val-0.pkl"
y_model_path = f"./LAMDA-TALENT/LAMDA_TALENT/results_model/{dataset_name}_Y-{model_name}-Tune/Norm-standard-Nan-mean-new-Cat-ordinal/best-val-0.pkl"

# === no Tune ===
#x_model_path = f"./LAMDA-TALENT/LAMDA_TALENT/results_model/{dataset_name}_X-{model_name}/Norm-standard-Nan-mean-new-Cat-ordinal/best-val-0.pkl"
#y_model_path = f"./LAMDA-TALENT/LAMDA_TALENT/results_model/{dataset_name}_Y-{model_name}/Norm-standard-Nan-mean-new-Cat-ordinal/best-val-0.pkl"

# cat indices
#x_model_path = f"./LAMDA-TALENT/LAMDA_TALENT/results_model/{dataset_name}_X-{model_name}-Tune/Norm-standard-Nan-mean-new-Cat-indices/best-val-0.pkl"
#y_model_path = f"./LAMDA-TALENT/LAMDA_TALENT/results_model/{dataset_name}_Y-{model_name}-Tune/Norm-standard-Nan-mean-new-Cat-indices/best-val-0.pkl"

# cat indices no Tune
#x_model_path = f"./LAMDA-TALENT/LAMDA_TALENT/results_model/{dataset_name}_X-{model_name}/Norm-standard-Nan-mean-new-Cat-indices/best-val-0.pkl"
#y_model_path = f"./LAMDA-TALENT/LAMDA_TALENT/results_model/{dataset_name}_Y-{model_name}/Norm-standard-Nan-mean-new-Cat-indices/best-val-0.pkl"


x_model = joblib.load(x_model_path)
y_model = joblib.load(y_model_path)

# === Detect if model uses GPU ===
use_gpu = Fals
try:
    use_gpu = (
        x_model.get_param('task_type').upper() == 'GPU' and
        y_model.get_param('task_type').upper() == 'GPU'
    )
except:
    pass  # Fallback to CPU timing if param not found

if model_name == "xgboost":
    use_gpu = True

print(f"device {use_gpu}")
print(x_model.get_params())

# === Inference Timing ===
if use_gpu:
    # GPU timing
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    starter.record()

    pred_x = x_model.predict(X_test_x) * std_x + mean_x
    pred_y = y_model.predict(X_test_y) * std_y + mean_y

    ender.record()
    torch.cuda.synchronize()
    inference_time = starter.elapsed_time(ender)  # milliseconds
else:
    # CPU timing
    start = time.time()

    pred_x = x_model.predict(X_test_x) * std_x + mean_x
    pred_y = y_model.predict(X_test_y) * std_y + mean_y

    end = time.time()
    inference_time = (end - start) * 1000  # convert to milliseconds
    
avg_time = inference_time / X_test_x.shape[0]

# === Combine predictions and compute metrics ===
preds = np.stack([pred_x, pred_y], axis=1)
true = np.stack([y_test_x, y_test_y], axis=1)

errors = np.sqrt(np.sum((preds - true) ** 2, axis=1))
mean_euclidean_error = np.mean(errors)

posx_rmse = np.sqrt(np.mean((preds[:, 0] - true[:, 0]) ** 2))
posy_rmse = np.sqrt(np.mean((preds[:, 1] - true[:, 1]) ** 2))

# === Print Results ===
print(f"Mean localization error (Euclidean distance): {mean_euclidean_error:.2f} mm")
print(f"RMSE PosX: {posx_rmse:.2f} mm")
print(f"RMSE PosY: {posy_rmse:.2f} mm")
print(f"Average inference time per sample: {avg_time:.3f} ms")


In [ ]:
import os

lines_to_add = [
    f"RMSE PosX: {posx_rmse}",
    f"RMSE PosY: {posy_rmse}",
    f"Mean Euclidean Error: {mean_euclidean_error}",
    f"Average Inference Time per Sample: {avg_time:.3f} ms\n"
]

# Define your path
metrics_path = f'logs/Regression/{dataset_name}/Classical/{model_name}/metrics.txt'

# Ensure directory exists
os.makedirs(os.path.dirname(metrics_path), exist_ok=True)
with open(metrics_path, 'a') as f:
    for line in lines_to_add:
        f.write(line + "\n")